In [1]:
from LabelEncoder import LabelEncoder
from sklearn.preprocessing import LabelEncoder as skLE
import numpy as np
import pandas as pd

In [17]:
y_train = ["paris", "paris", "tokyo", '-1', -1.0, -999, 'foo']
y_test1 = ["paris", "tokyo", "paris", -999, np.nan]
y_test2 = ["paris", "paris", "tokyo", "tel aviv", "amsterdam", -999, np.nan]

### Default fit transform is equivalent to sklearn label encoder. 

In [3]:
skle = skLE()

In [4]:
skle.fit_transform(y_train)

array([4, 4, 5, 0, 1, 2, 3])

In [5]:
skle.classes_

array(['-1', '-1.0', '-999', 'foo', 'paris', 'tokyo'],
      dtype='|S5')

In [6]:
le = LabelEncoder()

In [7]:
le.fit_transform(y_train)

array([0, 0, 1, 3, 2, 5, 4])

In [8]:
le.classes

0    paris
1    tokyo
2       -1
3       -1
4      foo
5     -999
dtype: object

In [9]:
le.most_common_class

'paris'

In [10]:
le.unknown_classes

[]

### Fit transform with nan classes defined and retained. 

In [18]:
le = LabelEncoder(retain_nan=True)

In [19]:
y_train_p = y_train
y_train_p.append(np.nan)

In [20]:
y_train_p

['paris', 'paris', 'tokyo', '-1', -1.0, -999, 'foo', nan]

In [21]:
le.fit_transform(y_train_p)

array([0, 0, 1, 3, 2, 5, 4, nan], dtype=object)

### Fit and transform with different train and test data

In [22]:
le = LabelEncoder(retain_nan=False)

In [23]:
le.fit(y_train)

In [24]:
le.classes

0    paris
1    tokyo
2       -1
3       -1
4      foo
5     -999
dtype: object

In [25]:
le.transform(y_test1)

LabelEncoder.py:50: UserWarning: y contains new labels!
  warnings.warn("y contains new labels!")


array([0, 1, 0, 5, 0])

In [27]:
le.unknown_classes

array([nan], dtype=object)

In [28]:
le.transform(y_test2)

array([0, 0, 1, 0, 0, 5, 0])

##

In [29]:
le = LabelEncoder(retain_nan=False)

In [30]:
le.fit(y_train)

In [31]:
le.transform(y_test1)

array([0, 1, 0, 5, 0])

In [32]:
le.transform(y_test2)

array([0, 0, 1, 0, 0, 5, 0])

In [33]:
le.test_classes

0        paris
1        tokyo
2     tel aviv
3    amsterdam
4         -999
5          NaN
dtype: object

In [34]:
le.classes

0    paris
1    tokyo
2       -1
3       -1
4      foo
5     -999
dtype: object

In [36]:
pd.Series([1,2,np.nan]).isin(pd.Series(np.nan))

0    False
1    False
2     True
dtype: bool